In [9]:
from textrazor import TextRazor
import re
import nltk
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

In [6]:
phraseDict = {'Jagdstaffel': ['Oswald Boelcke', 'Jagdstaffel', 'Jagdgeschwader 1 (World War I)', 'Dogfight', 'Bloody April', 'Synchronization gear', 'Aviation', 'World War I', 'Sopwith Camel', 'Fokker Dr.I', 'Max Immelmann', 'Anthony Fokker', 'Jagdstaffel 2', 'Cavalry', 'National security', 'Triplane', 'Aerial warfare', 'Jagdstaffel 11', 'Conflicts', 'Reconnaissance'], 'Air forces': ['Air forces', 'Air force', 'Royal Air Force'], 'No. 209 Squadron RAF': ['No. 209 Squadron RAF'], 'Military': ['Manfred von Richthofen', 'Fighter aircraft', 'Luftstreitkräfte', 'Cedric Popkin', 'Flying ace', 'Military', 'Military science', 'Military forces', 'Warfare', 'Military aviation', 'War', 'Aeronautics', 'Trench warfare', 'Aircraft', 'International security', 'Sopwith Aviation Company', 'Lanoe Hawker', 'Airplane', 'Aerospace engineering', 'Combat', 'Royal Flying Corps', 'Military organization', 'Violent conflict', 'Military operations'], 'Imperial German Army Air Service': ['Imperial German Army Air Service'], 'Roy Brown (RAF officer)': ['Roy Brown (RAF officer)']}

In [7]:
len(phraseDict)

6

In [10]:
## Functions Definitions ##

def get_top_relevant_topics(response, thresh_score):
    top_relevant_topics = []
    topics = []
    topics = response.topics()
    top_relevant_topics = [topic for topic in topics if topic.score > thresh_score]

    return top_relevant_topics

In [35]:
client = TextRazor("fdd39e9dde7d84983bbe964c514035f3c483d7df5996f62541ef76bc", extractors=["topics"])
i = 0
for sentence in sentences:
    topicDict = {}
    if(i < 1):
        print(sentence)
        response = client.analyze(sentence)
        topics = get_top_relevant_topics(response, 0.5)
        i += 1
        for topic in topics:
            print(" ", topic.label, topic.score)
            topicDict.update({topic.label: topic.score})
    else:
        break



NARRATOR: April 20th, 1918: In the skies over northern France, Allied and German fighter aircraft are locked in a ferocious dogfight.
  Aviation 1
  Military aviation 1
  Military forces 1
  Aerial warfare 1
  Aircraft 0.882
  Warfare 0.8187
  Aerial operations and battles 0.8173
  Military 0.7521
  Military science 0.7418
  War 0.7175
  Fighter aircraft 0.7161
  Combat 0.6892
  Aeronautics 0.6876
  Air-to-air combat operations and battles 0.644
  Military aircraft 0.6409
  Battles 0.6273
  Conflicts 0.6233
  Violent conflict 0.5761
  Wars 0.5647
  Military operations 0.5642
  Aerospace engineering 0.5398
  International security 0.5329


In [80]:
df = pd.DataFrame(index=phraseDict, columns=range(1,len(sentences)+1))
df = df.fillna(0) # with 0s rather than NaNs

In [136]:
df[1].iloc[5]

0.0

In [62]:
def buildArray(txtFile, clusterdict,windowsize):
    ####################Done once##############################################################################
    import math
    import csv
    from textrazor import TextRazorAnalysisException
    from urllib.error import HTTPError
    inFile = open(txtFile, encoding='utf8').read()
    tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
    sentences = tokenizer.tokenize(inFile)
    num_columns = math.ceil(len(sentences)/windowsize) + 1
    df = pd.DataFrame(index=phraseDict, columns=range(1,num_columns))
    df = df.fillna(0)
    
    
    sentence_num=1
    no_cluster_dict = {}
    textrazor_key = "eb2a5954577a28b1ba8a97005281289493eee48179d2ac1883b685e6"
    ###########################################################################################################
    
    for i in range(0,len(sentences),windowsize):
        #while i in range(0,len(sentences),windowsize):
        try:
            print("i = ", i)
            client = TextRazor(textrazor_key, extractors=["topics"])
            row=0
            sentence = str(sentences[i:i+windowsize])
            print(i , sentence)
            topicDict = {}
            response = client.analyze(sentence)
            topics = get_top_relevant_topics(response, 0.5)
            cluster_val = 0
            for topic in topics:
                print(" ", topic.label, topic.score)
                topicDict.update({topic.label: topic.score})
            for key in clusterdict:
                print('Phrase Number =',row)
                phrase = clusterdict.get(key)
                print(clusterdict.get(key))
                val = 0

                for topicKey in topicDict:
                    #print('Topic Number = ',j)
                    #print(topicKey, topicDict.get(topicKey))
                    if topicKey in phrase:
                        print(topicKey, topicDict.get(topicKey))
                        val += topicDict.get(topicKey)
                print('val =', val)
                print('row =', row, 'sentence_num =', sentence_num)
                df[sentence_num].iloc[row] = val
                cluster_val += val
                row+= 1
            if cluster_val == 0:
                no_cluster_dict.update({sentence_num: topicDict})
            sentence_num += 1

        except (HTTPError, TextRazorAnalysisException) as e:
            print("Error on " ,textrazor_key)
            textrazor_key = "fdd39e9dde7d84983bbe964c514035f3c483d7df5996f62541ef76bc"#"324337f35ea1876aae377a6f1191ab5a5b9aaced32f9a4e3634e74c5"
            print("i = ", i)
            #i = i - windowsize
            #print("i = ", i)
            print(e)
            continue
    df_csv_file = 'output_ws'+str(windowsize)+ '.csv'
    no_cluster_csv_file = 'no_cluster_dict_ws'+str(windowsize)+ '.csv'
    print("Output files:", df_csv_file, no_cluster_csv_file)
    df.to_csv(df_csv_file)
    with open(no_cluster_csv_file, 'w') as csv_file:
        writer_dict = csv.writer(csv_file)
        for key, value in no_cluster_dict.items():
            writer_dict.writerow([key, value])
    return df
            
        
    

In [ ]:
my_df2 = buildArray("textFile1", phraseDict, 2)

In [53]:
#print(len(sentences))
import math
num_columns = math.ceil(413/2) + 1
df = pd.DataFrame(index=phraseDict, columns=range(1,num_columns))
df = df.fillna(0)
df

,1,2,3,4,5,6,7,8,9,10,...,198,199,200,201,202,203,204,205,206,207
Jagdstaffel,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Air forces,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
No. 209 Squadron RAF,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Military,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Imperial German Army Air Service,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Roy Brown (RAF officer),0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [203]:
my_df1

,1,2,3,4,5,6,7,8,9,10,...,404,405,406,407,408,409,410,411,412,413
Jagdstaffel,3.2968,2.6372,3.9418,0,0,0.8530,0,2.0000,0,0,...,0,0,3.6574,0.000,0,0,0,0,0,1.1932
Air forces,0.0000,1.0000,0.0000,0,0,0.0000,0,0.0000,0,0,...,0,0,0.0000,0.000,0,0,0,0,0,0.0000
No. 209 Squadron RAF,0.0000,0.0000,0.0000,0,0,0.0000,0,0.0000,0,0,...,0,0,0.0000,0.000,0,0,0,0,0,0.0000
Military,10.6521,8.7453,6.7357,0,0,3.3882,0,4.5893,0,0,...,0,0,10.9413,0.549,0,0,0,0,0,1.8173
Imperial German Army Air Service,0.0000,1.0000,0.0000,0,0,0.0000,0,0.0000,0,0,...,0,0,0.0000,0.000,0,0,0,0,0,0.0000
Roy Brown (RAF officer),0.0000,0.0000,0.0000,0,0,0.0000,0,0.0000,0,0,...,0,0,0.0000,0.000,0,0,0,0,0,0.0000
